# Data Challenge - Πρόβλεψη Πληρότητας Πτήσεων

<b>Η παρούσα εργασία έγινε στα πλαίσια του μαθήματος "Εξόρυξη Γνώσης από Βάσεις Δεδομένων και τον Παγκόσμιο Ιστό", από τους φοιτητές Αναστάσιο Τσούκα ΑΜ: 3140213, Βασίλη Σταυριανουδάκη ΑΜ: 3140193 και Έκτορα Τυπάλδο ΑΜ: 3140216.</b>

Αρχικά πριν ξεκινήσουμε να γράφουμε κώδικα για την διαμόρφωση του προγράμματος, η πρώτη μας προσέγγιση ήταν να μελετήσουμε τα δεδομένα. Προσπαθήσαμε να επεξεργαστούμε τα δεδομένα με όσο το δυνατόν καλύτερο και πιο αποδοτικό τρόπο. Όπως θα δούμε και στην συνέχεια, παρατηρήσαμε ότι μερικά από τα features μας ήταν επικαλυπτόμενα με άλλα, ενώ από άλλα δημιουργήσαμε καινούργια.

Στην αρχή βάζουμε τα απαραίτητα import που θα μας χρειαστούνε, φορτώνουμε το excel train.csv που περιέχει τα δεδομένα μας και στον πίνακα y_train κρατάμε το feature PAX για να χρησιμοποιηθεί κατα την εκπαίδευση.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import csv
from sklearn.metrics import f1_score
import datetime

dataset = pd.read_csv('train.csv')
y_train = dataset[['PAX']]
y_train = np.ravel(y_train)

Συγκεκριμένα κρατώντας τις στήλες Departure,Arrival όπου περιέχουν τον κωδικό του κάθε αεροδρομίου είδαμε ότι δεν χρειαζόταν να κρατήσουμε τις CityDeparture και CityArrival καθώς ήταν επικαλυπτόμενα. Οι παραπάνω στήλες θα αφαιρεθούν στην συνέχεια του κώδικα.

Από τις στήλες LongitudeArrival, LatitudeArrival, LongitudeDeparture, LatitudeDeparture δημιουργήσαμε μια καινούργια στήλη την distance, η οποία είναι η απόσταση μεταξύ των δύο αεροδρομίων που αφορά η κάθε πτήση.

In [2]:
from math import sin, cos, sqrt, atan2, radians
R = 6373.0
dataset['dlon'] = np.radians(dataset['LongitudeArrival']) - np.radians(dataset.LongitudeDeparture)
dataset['dlat'] = np.radians(dataset['LatitudeArrival']) - np.radians(dataset.LatitudeDeparture)

dataset['a'] = np.sin(dataset["dlat"].astype(np.float64)/2)**2 + np.cos(np.radians(dataset.LatitudeDeparture).astype(np.float64)) * np.cos(np.radians(dataset['LatitudeArrival']).astype(np.float64)) * np.sin(dataset["dlon"].astype(np.float64)/2)**2
dataset['c'] = 2 *np.arctan2(np.sqrt(dataset['a']).astype(np.float64), np.sqrt(1 - dataset['a']).astype(np.float64)).astype(np.float64) #atan2 h arctan2
dataset['distance'] = R*dataset['c']

Επίσης από το DateOfDeparture δημιουργήσαμε τρείς καινούργιες στήλες, μια για την ημέρα, μία για τον μήνα και μια για το έτος, τα οποία όπως θα δούμε και στην συνέχεια μας βοήθησαν να παράξουμε περαιτέρω στήλες.

In [3]:
splitDate = dataset["DateOfDeparture"].str.split("-", n = 3, expand = True)
dataset['Day'] = splitDate[2]
dataset['Month'] = splitDate[1] 
dataset['Year'] = splitDate[0]
dataset['Month'] = dataset['Month'].astype(str).astype(int)
dataset['Day'] = dataset['Day'].astype(str).astype(int)
dataset['Year'] = dataset['Year'].astype(str).astype(int)

Ακόμα από την καινούργια στήλη Day που δημιουργήθηκε προηγουμένως, φτιάχνουμε ένα καινούργιο feature το οποίο είναι 0 αν είμαστε στις αρχές του μήνα, 1 αν είμαστε στο μέσο του μήνα και 2 αν βρισκόμαστε στο τέλος. Επίσης από την ίδια στήλη δημιουργήσαμε ένα ακόμα feature το DOW(day of week) που παίρνει τιμές από 0-6 ανάλογα με το ποια μέρα της εβδομάδας είναι.
Επιπλέον από το καινούργιο feature DOW προσθέσαμε μια νέα Boolean στήλη, την WEEKDAY η οποία είναι 1 αν είναι Σαββατοκύριακο ή 0 διαφορετικά. Τέλος από την στήλη Month δημιουργήσαμε την στήλη Season η οποία αναπαριστά τις 4 εποχές του χρόνου.

In [4]:
tempArr = np.zeros(dataset.shape[0])
for i in range(dataset.shape[0]):
    if(dataset.iloc[i]['Day'] > 10 and dataset.iloc[i]['Day'] <= 20  ):
        tempArr[i] = 1
    elif(dataset.iloc[i]['Day'] <= 31 and dataset.iloc[i]['Day'] > 20):
        tempArr[i] = 2
        
dataset['dayOfmonth'] = tempArr

Year = dataset['Year'].tolist()
Month = dataset['Month'].tolist()
Day = dataset['Day'].tolist() 
DOW = list(range(len(Year)))
for i in range(len(Year)):
    DOW[i] = datetime.datetime( Year[i], Month[i], Day[i] ).weekday()
DOWS = pd.Series(DOW)
dataset['DOW'] = DOWS.values

tempweek = np.zeros(dataset.shape[0])

for i in range(dataset.shape[0]):
    if(dataset.iloc[i]['DOW']==5 or dataset.iloc[i]['DOW']==6 ):
        tempweek[i] = 1
dataset['WEEKDAY'] = tempweek


tempArr=np.zeros(dataset.shape[0])
for i in range(dataset.shape[0]):
    if(dataset.iloc[i]['Month']>=11 and dataset.iloc[i]['Month']<=1):
        tempArr[i]=0 #xeimwnas
    elif(dataset.iloc[i]['Month']>=2 and dataset.iloc[i]['Month']<=4):
        tempArr[i]=1 # anoiksh
    elif (dataset.iloc[i]['Month']>=5 and dataset.iloc[i]['Month']<=7):
        tempArr[i]=2 # kalokairi
    elif (dataset.iloc[i]['Month']>=8 and dataset.iloc[i]['Month']<=10):
        tempArr[i]=3 # fthiniporo
dataset['Season']=tempArr

Σε αυτό το βήμα δημιουργούμε ένα καινούργιο feature το wtd/std_wtd, το οποίο προέκυψε από το WeeksToDeparture διά το std_wtd τα οποία διαπιστώσαμε ότι έχουν υψηλή συσχέτιση όπως θα δούμε παρακάτω.

In [5]:
dataset['wtd/std_wtd'] = dataset['WeeksToDeparture'] / dataset['std_wtd']

Σκεφτήκαμε ότι ενδεχομένως κατά τις περιόδους των διακοπών ή αργιών μπορεί να υπάρχει διαφοροποίηση στις τιμές PAX. Για αυτό το λόγο δημιουργήσαμε μια καινούργια Boolean στήλη, την holiday, η οποία είναι 1 αν είναι αργία αλλιώς είναι 0. Τέλος, αποφασίσαμε να ενώσουμε τις δύο στήλες Departure και Arrival δημιουργώντας την dep-arr στήλη.

In [6]:
dataset['DateOfDeparture'] = splitDate[0] + '-' + splitDate[1] + '-' + splitDate[2]

import holidays 
hol = holidays.CA() + holidays.US() + holidays.MX() + holidays.TAR()
h = np.zeros(dataset.shape[0])
for i in range(dataset.shape[0]):
    if(hol.get(dataset.iloc[i]['DateOfDeparture']) is not None):
        h[i] = 1

dataset['holiday'] = h       
dataset['dep-arr'] = dataset['Departure'].astype(str) + dataset['Arrival']

Για να δημιουργήσουμε νέα χαρακτηριστικά, να βρούμε επικαλυπτώμενα, καθώς και να βρούμε τυχών συσχετίσεις μεταξύ τους, μεταξύ άλλων χρησιμοποιήσαμε το παρακάτω διαγράμματα.

<img src="11.png">

<img src="22.png">

Με τις παρακάτω γραμμές αφήνουμε στον πίνακα dataset μόνο τα χαρακτηριστικά που θα χρησιμοποιήσουμε για την εκπαίδευση.

In [7]:
dataset.drop(dataset.columns[[0,2,3,4,6,7,8,9,10,11]], axis=1, inplace=True)
dataset.drop(dataset.columns[[2,3,4,5]], axis=1, inplace=True)
print(list(dataset))

['Departure', 'Arrival', 'distance', 'Day', 'Month', 'Year', 'dayOfmonth', 'DOW', 'WEEKDAY', 'Season', 'wtd/std_wtd', 'holiday', 'dep-arr']


Για τις κατηγορικές μεταβλητές χρησιμοποιούμε LabelEncoder για την μετατροπή των αλφαριθμητικών σε αριθμούς. 

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['Departure'] = le.fit_transform(dataset['Departure'])
dataset['Arrival'] = le.fit_transform(dataset['Arrival'])
dataset['dep-arr'] = le.fit_transform(dataset['dep-arr'])

Από τις ποσοτικές στήλες που είχαμε, η μοναδική η οποία είχε μεγάλες διακυμάνσεις στις τιμές ήταν η στήλη distance, στην οποία κάναμε normalize για να μεταφέρουμε τις τιμές της στο διάστημα [0,1].

In [9]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
x = dataset[['distance']]
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
dataset['distance'] = x_scaled

<font size="4"><b>Αντίστοιχα ότι κάναμε για την προετοιμασία των δεδομένων που χρησιμοποιούμε για train, κάνουμε και για τα δεδομένα του test.</b></font> 

In [10]:
df_test = pd.read_csv('test.csv')
#########################################
R = 6373.0
df_test['dlon'] = np.radians(df_test['LongitudeArrival']) - np.radians(df_test.LongitudeDeparture)
df_test['dlat'] = np.radians(df_test['LatitudeArrival']) - np.radians(df_test.LatitudeDeparture)

df_test['a'] = np.sin(df_test["dlat"].astype(np.float64)/2)**2 + np.cos(np.radians(df_test.LatitudeDeparture).astype(np.float64)) * np.cos(np.radians(df_test['LatitudeArrival']).astype(np.float64)) * np.sin(df_test["dlon"].astype(np.float64)/2)**2
df_test['c'] = 2 *np.arctan2(np.sqrt(df_test['a']).astype(np.float64), np.sqrt(1 - df_test['a']).astype(np.float64)).astype(np.float64) #atan2 h arctan2
df_test['distance'] = R*df_test['c']
###########################################
splitDate2 = df_test["DateOfDeparture"].str.split("-", n = 3, expand = True)
df_test['Day'] = splitDate2[2]
df_test['Month'] = splitDate2[1] 
df_test['Year'] = splitDate2[0]
df_test['Month'] = df_test['Month'].astype(str).astype(int)
df_test['Day'] = df_test['Day'].astype(str).astype(int)
df_test['Year'] = df_test['Year'].astype(str).astype(int)

tempArr=np.zeros(df_test.shape[0])
for i in range(df_test.shape[0]):
    if(df_test.iloc[i]['Day']>10 and df_test.iloc[i]['Day']<=20  ):
        tempArr[i]=1
    elif(df_test.iloc[i]['Day']<=31 and df_test.iloc[i]['Day']>20):
        tempArr[i]=2
        
df_test['dayOfmonth']=tempArr        

Year = df_test['Year'].tolist()
Month = df_test['Month'].tolist()
Day = df_test['Day'].tolist() 
DOW = list(range(len(Year)))
for i in range(len(Year)):
    DOW[i] = datetime.datetime( Year[i], Month[i], Day[i] ).weekday()
DOWS = pd.Series(DOW)
df_test['DOW'] = DOWS.values
###################################################################
tempweek = np.zeros(df_test.shape[0])

for i in range(df_test.shape[0]):
    if(df_test.iloc[i]['DOW']==5 or df_test.iloc[i]['DOW']==6 ):
        tempweek[i] = 1
df_test['WEEKDAY'] = tempweek

tempArr=np.zeros(df_test.shape[0])
for i in range(df_test.shape[0]):
    if(df_test.iloc[i]['Month']>=11 and df_test.iloc[i]['Month']<=1):
        tempArr[i]=0 #xeimwnas
    elif(df_test.iloc[i]['Month']>=2 and df_test.iloc[i]['Month']<=4):
        tempArr[i]=1 # anoiksh
    elif (df_test.iloc[i]['Month']>=5 and df_test.iloc[i]['Month']<=7):
        tempArr[i]=2 # kalokairi
    elif (df_test.iloc[i]['Month']>=8 and df_test.iloc[i]['Month']<=10):
        tempArr[i]=3 # fthiniporo
df_test['Season']=tempArr
###############################################################
df_test['wtd/std_wtd'] = df_test['WeeksToDeparture'] / df_test['std_wtd']

df_test['DateOfDeparture'] = splitDate2[0] + '-' + splitDate2[1] + '-' + splitDate2[2]

import holidays 
hol = holidays.CA() + holidays.US() + holidays.MX() + holidays.TAR()
h = np.zeros(df_test.shape[0])
for i in range(df_test.shape[0]):
    if(hol.get(df_test.iloc[i]['DateOfDeparture']) is not None):
        h[i] = 1

df_test['holiday'] = h 

df_test.drop(df_test.columns[[0,2,3,4,6,7,8,9,10]], axis=1, inplace=True)
df_test.drop(df_test.columns[[2,3,4,5]], axis=1, inplace=True)   

df_test['dep-arr'] = df_test['Departure'].astype(str) + df_test['Arrival']


le = LabelEncoder()
df_test['Departure'] = le.fit_transform(df_test['Departure'])
df_test['Arrival'] = le.fit_transform(df_test['Arrival'])
df_test['dep-arr'] = le.fit_transform(df_test['dep-arr'])
#####################################################normalize
from sklearn import preprocessing
x = df_test[['distance']]
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_test['distance'] = x_scaled
#########################################

Σε περίπτωση που θέλουμε να τρέξουμε τον κώδικα τοπικά για δοκιμές, αντί για το παραπάνω κομμάτι κώδικα, χρησιμοποιούμε αυτόν εδώ:

In [11]:
#from sklearn.cross_validation import train_test_split
#dataset, df_test, y_train, y_test = train_test_split(dataset, y_train, test_size=0.2, random_state=42)

Όπως βλέπουμε στην παρακάτω φωτογραφία το PAX 4 έχει ελάχιστα στοιχεία σε σχέση με τα υπόλοιπα, δηλαδή τα δεδομένα μας είναι imbalanced και αυτό έχει ως αποτέλεσμα σχεδόν να παραβλέπεται από τους αλγορίθμους εκπαίδευσης. Για τον λόγο αυτό, χρησιμοποιήσαμε την τεχνική SMOTE (Synthetic Minority Over-sampling Technique) για να παράξουμε νέα συνθετικά δεδομένα με PAX 4. Στην συνέχεια επειδή τα δεδομένα που παράγονται βγαίνουν κατά προσέγγιση από τα πραγματικά δεδομένα, αυτό έχει ως αποτέλεσμα ορισμένες τιμές να μην είναι ακέραιες και γι’ αυτό το λόγο τους κάνουμε στρογγυλοποίηση.

<img src="33.png">

In [12]:
shapeTrain = dataset.shape[0]
cols = list(dataset)        
from imblearn.over_sampling import SMOTE
smote = SMOTE(ratio='minority')
dataset, y_train = smote.fit_sample(dataset, y_train)

shapeSmote = dataset.shape[0]

for i in range(round((shapeSmote - shapeTrain) * 5/6)):
    dataset = dataset[:-1]
    y_train = y_train[:-1]

dataset = pd.DataFrame(dataset)

dataset.columns = cols
############################### Round up
for i in range(0 , dataset.shape[0]):
    for j in range(0 , dataset.shape[1]):
        if(dataset.columns[j] != 'distance' and dataset.columns[j] != 'wtd/std_wtd'):            
            dataset.iloc[i,j] = round(dataset.iloc[i,j])    

Στην συνέχεια περνάμε όλα μας τα δεδομένα από τον OneHotEncoder.

In [13]:
from sklearn.preprocessing import OneHotEncoder
X_train = dataset
X_test = df_test
enc = OneHotEncoder(sparse=False)
enc.fit(dataset)  
X_train = enc.transform(dataset)
X_test = enc.transform(df_test)

Σαν τελικό μοντέλο επιλέξαμε το παρακάτω νευρωνικό δίκτυο για την εκπαίδευση των δεδομένων το οποίο αποτελείται από:

-Το πρώτο layer χρησιμοποιεί την activation function “relu” και δέχεται σαν είσοδο όλες τις στήλες του πίνακα X_train.

-Το δεύτερο και το τρίτο layer χρησιμοποιούν πάλι την activation function “relu” και μετά από δοκιμές προσαρμόσαμε τους κόμβους τους σε 190 και 80 αντίστοιχα.

-Για να αποφύγουμε το overfitting, προσθέσαμε για κάθε hidden layer Dropout 0.30.

-Έπειτα, στο τελευταίο επίπεδο του δικτύου το οποίο έχει 8 εξόδους (μία για κάθε κλάση), χρησιμοποιήσαμε την  activation function “softmax” ώστε να μπορεί να γίνει πολλαπλή κατηγοριοποίηση. 

-Τέλος, κάναμε compile το μοντέλο μας με optimizer “rmsprop” ο οποίος μετά από δοκιμές μας έβγαζε τα καλύτερα αποτελέσματα.


In [14]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
#Initializing Neural Network
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(kernel_initializer="uniform", activation="relu", input_dim=X_train.shape[1], units=(190)))
###gia to overfitting
from keras.layers import Dropout
classifier.add(Dropout(0.30))

# Adding the second hidden layer
classifier.add(Dense(kernel_initializer="uniform", activation="relu", units=(80)))
###gia to overfitting
classifier.add(Dropout(0.30))

# Adding the output layer
classifier.add(Dense(kernel_initializer="uniform", activation="softmax", units=8))


# Compiling Neural Network
classifier.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

C:\Users\Ironik\Anaconda3\envs\DeepLeanring\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Επειδή το τελευταίο layer του νευρωνικού δικτύου αποτελείται από 8 εξόδους (μία για κάθε PAX) πρέπει να μετατρέψουμε τα δεδομένα του y_train που θα δοθούν για την εκπαίδευση σε one hot τιμές.

Στην συνέχεια εκπαιδεύουμε το μοντέλο μας με 30 εποχές (έπειτα από δοκιμές) και τα αποτελέσματα τα μετατρέπουμε από one hot τιμές σε κανονικά PAX με τιμές από 0-7.


In [15]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(encoded_Y)
# Fitting our model 
classifier.fit(X_train, dummy_y,  epochs = 30)
# Predicting the Test set results
y_pred = classifier.predict(X_test)

yPRED = np.zeros(y_pred.shape[0])
for i in range(y_pred.shape[0]):
    yPRED[i]=np.argmax(y_pred[i])

Epoch 1/30
9240/9240 [==============================] - 1s 95us/step - loss: 1.6939 - acc: 0.2898
Epoch 2/30
9240/9240 [==============================] - 0s 49us/step - loss: 1.3384 - acc: 0.4275
Epoch 3/30
9240/9240 [==============================] - 0s 49us/step - loss: 1.2252 - acc: 0.4846
Epoch 4/30
9240/9240 [==============================] - 0s 47us/step - loss: 1.1533 - acc: 0.5077
Epoch 5/30
9240/9240 [==============================] - 0s 47us/step - loss: 1.0979 - acc: 0.5320
Epoch 6/30
9240/9240 [==============================] - 0s 46us/step - loss: 1.0622 - acc: 0.5527
Epoch 7/30
9240/9240 [==============================] - 0s 51us/step - loss: 1.0307 - acc: 0.5673
Epoch 8/30
9240/9240 [==============================] - 0s 47us/step - loss: 0.9906 - acc: 0.5776
Epoch 9/30
9240/9240 [==============================] - 0s 50us/step - loss: 0.9695 - acc: 0.5937
Epoch 10/30
9240/9240 [==============================] - 0s 46us/step - loss: 0.9397 - acc: 0.6026
Epoch 11/30
9240/92

Τέλος, δημιουργούμε το αρχείο για το submission στο Kaggle.

In [16]:
with open('y_pred.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Id', 'Label'])
    for i in range(yPRED.shape[0]):
        writer.writerow([i, yPRED[i].astype(np.int)])  

Τον παρακάτω κώδικα τον χρησιμοποιούμε για να βγάλουμε τοπικά το f1 score καθώς και για να φτιάξουμε τον confusion matrix που μας βοηθάει να δούμε τις πραγματικές τιμές του κάθε PAX σε σχέση με τις προβλέψεις.

In [17]:
#print(f1_score(y_test, yPRED, average='micro'))
#from sklearn.metrics import confusion_matrix 
#cm = confusion_matrix(y_test, yPRED)

Με τον παραπάνω αλγόριθμο εκπαίδευσης έχουμε μια απόδοση f1 score ανάμεσα στο 60-64% και συνολικά χρειάζεται περίπου 80 δευτερόλεπτα για να ολοκληρωθεί.

# Αλγόριθμοι που δοκιμάσαμε κατα την ανάπτυξη της εργασίας

Βέβαια, πριν ασχοληθούμε με τα νευρωνικά δίκτυα δοκιμάσαμε και άλλους αλγόριθμους ταξινόμησης όπως:

(Στους πρώτους τρείς αλγορίθμους που θα σας παρουσιάσουμε η επεξεργασία των δεδομένων μας ήταν σε αρχικό στάδιο ακόμα και αυτός ίσως ήταν ένας λόγος όπου τα σκορ τους δεν είχαν σημαντικές βελτιώσεις)

- K-nearest neighbors: ο συγκεκριμένος αλγόριθμος ήταν ο πρώτος που χρησιμοποιήσαμε και με διάφορες βελτιώσεις καταφέραμε να φτάσουμε το σκορ 42%.

- Naïve Bayes: στην συνέχεια δοκιμάσαμε τον συγκεκριμένο αλγόριθμο αλλά για τον υπολογισμό των πιθανοτήτων δεν χρησιμοποιήσαμε έτοιμη συνάρτηση αλλά φτιάξαμε την υλοποίηση μόνοι μας. Το παραπάνω όμως δεν κατάφερε να βελτιώσει σημαντικά το σκορ.

- Ensemble learning (stacking): Είχαμε την ιδέα να συνδυάσουμε περισσότερους από έναν αλγορίθμους έτσι ώστε να πετύχουμε κάποια βελτίωση στο σκορ. Στην συγκεκριμένη μέθοδο χωρίσαμε το dataset σε πέντε κομμάτια και εκπαιδεύσαμε πέντε διαφορετικά μοντέλα με κάθε ένα από αυτά. Στην συνέχεια, συνδυάσαμε τις προβλέψεις των πρώτων μοντέλων δίνοντάς τες στο επόμενο επίπεδο του αλγορίθμου που χρησιμοποιούσε το μοντέλο xgboost με σκοπό να βγάλει μια καλύτερη πρόβλεψη μαθαίνοντας από τις προηγούμενες. Βέβαια, δεν πετύχαμε ούτε εδώ κάποια σημαντική βελτίωση.

(Σε αυτό το σημείο η επεξεργασία των δεδομένων μας είχε προχωρήσει πολύ και έτσι εκτός από την επιλογή του κατάλληλου μοντέλου κάναμε περισσότερες αλλαγές και επεξεργασία στα δεδομένα)

- Logistic Regression: Έπειτα από την επεξεργασία των δεδομένων μας, για να δούμε αν οι αλλαγές μας βοηθάνε στις προβλέψεις θέλαμε να χρησιμοποιήσουμε έναν σχετικά απλό αλγόριθμο και γι’ αυτό τον λόγο τον επιλέξαμε. Παρατηρήσαμε ότι η επεξεργασία των δεδομένων μας όντως βοηθούμε καθώς με αυτόν τον αλγόριθμο είχαμε σκορ 49-50% το οποίο ήταν μια σημαντική βελτίωση σε σχέση με τους προηγούμενους αλγορίθμους.

- GradientBoostingClassifier: Παρατηρήσαμε από το stacking ότι ο συγκεκριμένος αλγόριθμος ξεχώριζε σε σχέση με τους άλλους και γι’ αυτό τον λόγο αποφασίσαμε να τον δοκιμάσουμε με τα νέα δεδομένα. Έτσι καταφέραμε να βελτιώσουμε άλλο λίγο το σκορ και να το φτάσουμε 52-53%

- Νευρωνικά Δίκτυα: Σε αυτό το σημείο αφού είμασταν σίγουροι για την επεξεργασία των δεδομένων μας προσπαθήσαμε να διαλέξουμε έναν καλύτερο αλγόριθμο, επιλέγοντας τα νευρωνικά δίκτυα. Ο συγκεκριμένος αλγόριθμος εκπαίδευσης μετά από πολλές δοκιμές για τις παραμέτρους του έφτασε να έχει το καλύτερο σκορ από όλους και γι’ αυτό τον λόγο τον κρατήσαμε.

 
